## 🦠 **Indicadores Epidemiológicos**
### 1. **Total de casos por município**
- Soma total de casos em cada cidade.

### 2. **Incidência acumulada por 100 mil habitantes**
- Fórmula: `(casos / população) * 100000`
- Mostra a taxa de infecção proporcional — muito útil para comparar cidades de tamanhos diferentes.

### 3. **Casos por semana epidemiológica (SE)**
- Ajuda a detectar surtos, aumentos repentinos e tendências.

### 4. **Classificação de risco (nível) por município**
- Quantidade de cidades em cada categoria de risco: *baixo*, *médio* ou *alto*.

### 5. **Variação percentual de casos semana a semana**
- Para medir o crescimento ou diminuição da dengue ao longo do tempo.

---

## 🌡️ **Indicadores Climáticos**
> Muito úteis porque o clima influencia diretamente na proliferação do mosquito.

### 6. **Média de temperatura mínima, média e máxima por município**
- Pode ser cruzado com o número de casos para investigar correlações.

### 7. **Média de umidade mínima, média e máxima por município**
- Umidade elevada favorece o mosquito, então é outro ponto a ser analisado.

---

## 📊 **Indicadores Combinados (Clima + Casos)**
### 8. **Correlação entre temperatura média e casos**
- Use `df.corr()` ou métodos estatísticos para descobrir se existe relação.

### 9. **Correlação entre umidade e casos**

### 10. **Semana epidemiológica com maior número de casos por cidade**
- Mostra o pico da doença por local.

---

## 🌍 **Indicadores Geográficos (com uso de GeoPandas)**
### 11. **Mapa de calor de incidência**
- Usando o `gdf` com `base_compilada`, dá pra fazer um mapa com `folium` ou `geopandas.plot()`.

### 12. **Top 10 municípios com maior incidência**
- Ajuda a priorizar ações em campo.

---

## 🛠️ **Outros KPIs Estratégicos**
### 13. **Número de municípios com risco alto**
- Pode virar uma métrica de alerta para o governo/órgãos de saúde.

### 14. **Dias ou semanas consecutivas em risco alto**
- Um indicador de persistência do surto.

### 15. **Razão entre casos e população (densidade de casos)**
- Complementa o KPI de incidência.

In [1]:
import pandas as pd
import numpy as np

base = pd.read_csv('C:/Users/ree11390/OneDrive - M DIAS BRANCO S.A. INDUSTRIA E COMERCIO DE ALIMENTOS/Área de Trabalho/TRABALHO FACULDADE/bases/base_compilada.csv', sep=";")

**INDICADORES EPIDEMIOLÓGICOS**

In [2]:
# 1 - TOTAL DE CASOS POR MUNÍCIPIO
kpi_casos_municipio = base.groupby('municipios')['casos'].sum().reset_index()
kpi_casos_municipio.columns = ['municipios', 'total_casos']
#kpi_casos_municipio = kpi_casos_municipio.sort_values(by='total_casos', ascending=False).reset_index(drop=True)

# 2 - INCIDÊNCIA ACUMLADA POR 100 MIL HABITANTES
kpi_100_mil_habitantes = base.groupby('municipios')['pop'].sum().reset_index()
kpi_100_mil_habitantes['incidencia_100k'] = (kpi_casos_municipio['total_casos']/ kpi_100_mil_habitantes['pop']) * 100_00

base_kpi_atemporal = pd.merge(kpi_casos_municipio, kpi_100_mil_habitantes, on='municipios')

# 3 - CASOS POR SEMANA EPIDEMIOLÓGICA
kpi_casos_semana = base.groupby(['SE', 'municipios'])['casos'].sum().reset_index()
kpi_casos_semana.columns = ['semana_epidemiologica', 'municipios', 'total_casos'] 

base_kpi_temporal = kpi_casos_semana

# 4 - CLASSIFICAÇÃO DE RISCO (NÍVEL) POR MUNCÍPIO
base_ordenada = base.sort_values(by='data_iniSE', ascending=False)
nivel_atual = base_ordenada.drop_duplicates(subset=['municipios'])[['municipios', 'nivel']]
distribuicao_risco = nivel_atual['nivel'].value_counts().reset_index()
distribuicao_risco.columns = ['nivel_risco', 'quantidade_cidades']
kpi_classificacao_risco_municipio = distribuicao_risco

# 5 - VARIAÇÃO PERCETUAL DE CASOS SEMANA A SEMANA
casos_semana = base.groupby('data_iniSE')['casos'].sum().reset_index()
kpi_variacao_percent_semana_semana = casos_semana.sort_values('data_iniSE')
kpi_variacao_percent_semana_semana['variacao_percentual'] = casos_semana ['casos'].pct_change() * 100
kpi_variacao_percent_semana_semana['variacao_percentual'] = kpi_variacao_percent_semana_semana['variacao_percentual'].round(2)

**INDICADORES CLIMÁTICOS**

In [3]:
# 6 - MÉDIA DE TEMPERATURA MÍNIMA, MÉDIA E MÁXIMA POR MUNICÍPIO
kpi_media_temperatura_municipio = base.groupby('municipios')[['tempmin', 'tempmed', 'tempmax']].mean().reset_index()
kpi_media_temperatura_municipio.columns = ['municipios', 'media_temp_min', 'media_temp_media', 'media_temp_max']

base_kpi_atemporal = base_kpi_atemporal.merge(kpi_media_temperatura_municipio, on='municipios', how='left')

# 7 - MÉDIA DE UMIDADE MÍNIMA, MÉDIA E MÁXIMA POR MUNICÍPIO
kpi_media_umidade_municipio = base.groupby('municipios')[['umidmin', 'umidmed', 'umidmax']].mean().reset_index()
kpi_media_umidade_municipio.columns = ['municipios', 'media_umidade_mini', 'media_umidade_media', 'media_umidade_max']

base_kpi_atemporal = base_kpi_atemporal.merge(kpi_media_umidade_municipio, on='municipios', how='left')

**INDICADORES COMBINADOS (CLIMA + CASOS)**

In [4]:
# 8 - CORRELAÇÃO ENTRE TEMPERATURA MÉDIA E CASOS
def calcular_correlacao_temp(grupo):
    return grupo['tempmed'].corr(grupo['casos'])

kpi_correlacao_temp_media_casos = base.groupby('municipios').apply(calcular_correlacao_temp).reset_index()
kpi_correlacao_temp_media_casos.columns = ['municipios', 'corrrelacao_tempmed_casos']

base_kpi_atemporal = base_kpi_atemporal.merge(kpi_correlacao_temp_media_casos, on='municipios', how='left')

# 9 - CORRELAÇÃO ENTRE UMIDADE E CASOS
def calcular_correlacao_umd(grupo):
    return grupo['umidmed'].corr(grupo['casos'])

kpi_correlacao_umid_media_casos = base.groupby('municipios').apply(calcular_correlacao_umd).reset_index()
kpi_correlacao_umid_media_casos.columns = ['municipios', 'kpi_correlacao_umid_media_casos']

base_kpi_atemporal = base_kpi_atemporal.merge(kpi_correlacao_umid_media_casos, on='municipios', how='left')

# 10 - SEMANA EPIDEMIOLÓGICA COM MAIOR NÚMERO DE CASOS POR CIDADE
kpi_semana_maior_num_casos_cidade = base.loc[
    base.groupby('municipios')['casos'].idxmax(),
    ['municipios', 'SE', 'casos', 'data_iniSE']
].rename(columns={
    'SE': 'semana_pico',
    'casos': 'casos_pico',
    'data_iniSE': 'data_inicio_pico'
})

base_kpi_atemporal = base_kpi_atemporal.merge(kpi_semana_maior_num_casos_cidade, on='municipios', how='left')    

C:\Users\ree11390\AppData\Local\Temp\ipykernel_15896\227862188.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  kpi_correlacao_temp_media_casos = base.groupby('municipios').apply(calcular_correlacao_temp).reset_index()
C:\Users\ree11390\AppData\Local\Temp\ipykernel_15896\227862188.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  kpi_correlacao_umid_media_casos = base.groupby('municipios').apply(calcu

**OUTROS KPIs ESTRATÉGICOS**

In [5]:
# 11 - NÚMERO DE MUNICÍPIOS COM RISCO ALTO (QUALQUER SEMANA)
kpi_municipios_risco_alto_qqsemana = base[base['nivel'] == 4]['municipios'].unique()

base_kpi_atemporal['teve_risco_alto'] = base_kpi_atemporal['municipios'].isin(kpi_municipios_risco_alto_qqsemana)

# 12 - NÚMERO DE MUNICÍPIOS COM RISCO ALTO (ÚLTIMA SEMANA)
ultima_semana = base['SE'].max()
kpi_municipios_risco_alto_ultisemana = base[
    (base['SE'] == ultima_semana) &
    (base['nivel'] == 4)
]['municipios'].unique()

base_kpi_atemporal['teve_risco_alto_ultima_semana'] = base_kpi_atemporal['municipios'].isin(kpi_municipios_risco_alto_ultisemana)

# 13 - SEMANAS CONSECUTIVAS EM RISCO ALTO
base_ordenada = base.sort_values(by=['municipios', 'SE'])
base_ordenada['risco_alto'] = (base_ordenada['nivel'] == 4).astype(int)
base_ordenada['quebra'] = (
    (base_ordenada['risco_alto'] != base_ordenada.groupby('municipios')['risco_alto'].shift()) |
    ((base_ordenada['SE'] != base_ordenada.groupby('municipios')['SE'].shift() + 1) &
     (base_ordenada['risco_alto'] == 1))
).cumsum()

sequencias = base_ordenada[base_ordenada['risco_alto'] == 1].copy()
sequencias['tamanho_sequencia'] = sequencias.groupby(['municipios', 'quebra'])['risco_alto'].transform('count')

kpi_persistencia = sequencias.groupby('municipios')['tamanho_sequencia'].max().reset_index()
kpi_persistencia.columns = ['municipios', 'max_semanas_consec_risco_alto']

base_kpi_atemporal = base_kpi_atemporal.merge(kpi_persistencia, on='municipios', how='left')
base_kpi_atemporal['max_semanas_consec_risco_alto'] = base_kpi_atemporal['max_semanas_consec_risco_alto'].fillna(0).astype(int)

# 14 - RAZÃO ENTRE CASOS E POPULAÇÃO (DENSIDADE DE CASOS)
base_kpi_atemporal['densidade_casos'] = (base_kpi_atemporal['total_casos'] / base_kpi_atemporal['pop'] *100).round(2)

In [8]:
base_kpi_atemporal.to_json("C:/Users/ree11390/OneDrive - M DIAS BRANCO S.A. INDUSTRIA E COMERCIO DE ALIMENTOS/Área de Trabalho/TRABALHO FACULDADE/bases/kpis_atemporais.json", 
                           orient='records', 
                           force_ascii=False
)

base_kpi_temporal.to_json("C:/Users/ree11390/OneDrive - M DIAS BRANCO S.A. INDUSTRIA E COMERCIO DE ALIMENTOS/Área de Trabalho/TRABALHO FACULDADE/bases/kpis_temporais.json", 
                          orient='records', 
                          force_ascii=False
)

kpi_classificacao_risco_municipio.to_json("C:/Users/ree11390/OneDrive - M DIAS BRANCO S.A. INDUSTRIA E COMERCIO DE ALIMENTOS/Área de Trabalho/TRABALHO FACULDADE/bases/kpi_classificacao_risco_municipio.json", 
                                          orient='records', 
                                          force_ascii=False
)

kpi_variacao_percent_semana_semana.to_json("C:/Users/ree11390/OneDrive - M DIAS BRANCO S.A. INDUSTRIA E COMERCIO DE ALIMENTOS/Área de Trabalho/TRABALHO FACULDADE/bases/kpi_variacao_percent_semana_semana.json", 
                                           orient='records', 
                                           force_ascii=False
)